In [1]:
from transformers import AutoModel, AutoTokenizer

model_name = "BAAI/bge-base-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/home/luis/anaconda3/envs/STAR/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
text = "Your sample text goes here."

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

import torch

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)

print(embeddings)

tensor([[ 9.4439e-01,  5.7841e-01,  3.4593e-01,  1.0775e+00,  8.5433e-01,
          2.9458e-01,  7.2219e-01,  6.7924e-01, -8.4592e-02, -8.2538e-01,
          2.0097e-01,  3.4050e-01, -1.1064e+00, -3.4523e-01, -4.1352e-01,
          7.1441e-01,  5.9369e-01,  5.0657e-01,  5.3565e-02, -3.2880e-01,
         -1.7285e-01,  7.3843e-01,  7.8320e-01,  5.9878e-01,  5.4478e-01,
          7.2674e-04,  4.7730e-01,  9.8830e-02, -1.1201e+00,  1.8203e-01,
          2.9416e-01, -1.3657e-01,  3.7110e-02, -6.2774e-01, -2.9116e-01,
         -2.9002e-01,  9.6554e-02, -3.6251e-01, -2.1456e-02, -1.4774e-02,
         -7.5330e-01,  3.0531e-01, -3.0068e-01, -1.6493e-01, -2.8731e-01,
          1.0875e-01, -2.1382e-01,  1.9427e-01, -7.1262e-01,  3.0448e-01,
         -5.0007e-01,  6.8167e-01,  1.2404e-01,  2.2038e-01, -5.5693e-01,
          7.6559e-01, -1.4807e-01, -7.0849e-01,  2.6368e-02, -6.4200e-01,
         -1.4524e-01,  1.2407e-01,  1.3725e-01,  4.8076e-01,  4.9819e-01,
          2.5411e-01,  6.1374e-01,  6.

In [19]:
text2 = "Je m'apelle François"

inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

import torch

with torch.no_grad():
    outputs = model(**inputs2)
    embeddings2 = outputs.last_hidden_state.mean(dim=1)

print(embeddings2)    

tensor([[-3.4056e-01, -7.2067e-02,  3.0589e-01,  1.8849e-01,  5.4942e-01,
          5.3856e-01,  1.0997e+00,  2.6053e-01, -5.7295e-01, -1.0898e+00,
         -3.5385e-01,  1.9115e-01, -4.9280e-01, -7.4668e-02, -1.5357e-02,
          6.5124e-01,  7.9564e-01, -4.7632e-01, -1.0742e-01, -2.6815e-01,
          2.3470e-01,  4.3147e-01,  9.7983e-01,  5.4741e-01, -2.2478e-01,
          3.9870e-01,  5.7722e-02,  7.5463e-01, -6.0933e-01, -2.7534e-01,
          5.5484e-01,  2.0322e-01,  1.6511e-01, -1.2516e+00, -1.9584e-01,
         -1.9151e-01,  9.6600e-01, -1.4505e-01, -3.6979e-01,  3.1511e-01,
          5.2499e-02, -6.6708e-01,  2.0181e-02, -2.2265e-01, -1.0385e+00,
         -4.2920e-01,  6.7953e-02,  1.0406e-01, -4.7698e-01, -5.6671e-01,
         -3.3687e-01,  5.8362e-01,  6.0036e-01,  2.8016e-03, -4.3562e-01,
          5.7104e-01, -6.3776e-01, -8.6332e-01, -2.8901e-01, -3.5021e-01,
          3.0697e-01,  2.3805e-01,  1.1865e+00,  2.5204e-01, -2.0966e-01,
          1.7720e-01, -5.0981e-01,  3.

In [20]:
import torch

# Assuming embeddings and embeddings2 are tensors of shape (1, hidden_dim)
cosine_similarity = torch.nn.functional.cosine_similarity(embeddings, embeddings2, dim=1)

# Print the similarity score
print("Cosine Similarity:", cosine_similarity.item())

Cosine Similarity: 0.5604743957519531
